In [22]:
from collections import deque
import sys
sys.setrecursionlimit(100000000)
import gzip
import pickle
from multiprocessing import Pool
import numpy as np
import time

In [42]:
def SCC(adj):
    n = len(adj)
    S = []
    SCCs = []
    pre = [0]*n
    post = [0]*n
    c = 0

    def dfs(u):
        nonlocal c
        c+=1
        pre[u]=c
        S.append(u)
        minpre = pre[u]
        for v in adj[u]:
            if not pre[v]: minpre = min(minpre, dfs(v))
            elif not post[v]: minpre = min(minpre, pre[v])
        if minpre == pre[u]:
            SCCs.append([])
            while True:
                v = S.pop()
                SCCs[-1].append(v)
                post[v]=1
                if v==u: break
        return minpre

    for i in range(n):
        if not pre[i]: dfs(i)

    return SCCs


def get_contam(adj):
    n = len(adj)
    SCCs = SCC(adj)
    contam = 0

    def reach_num(u):
        visit = [0]*n
        visit[u] = 1
        Q = deque([u])
        while Q:
            u = Q.popleft()
            for v in adj[u]:
                if not visit[v]:
                    visit[v] = 1
                    Q.append(v)
        return sum(visit)

    for scc in SCCs: contam += reach_num(scc[0])*len(scc)
    contam /= n

    return contam


def BPM(adj_list, seed_idx, del_edge_num, sampling_num=10000):
    time_start = time.time()
    time_alg = []
    mask = []
    
    # make edge list for efficient sampling
    n = len(adj_list)
    edge = []
    for u in range(n):
        for v,p in adj_list[u]: edge.append([u,v,p])
    edge = np.array(edge)
    m = len(edge)

    for _ in range(del_edge_num):        
        # sampling
        samplings = np.random.rand(sampling_num,m)
        live_edges = samplings<edge[:,2]
        dead_edges = ~live_edges
    
        # calculate contamination degree for each sample
        def make_adj(live_edge):
            adj = [[] for _ in range(n)]
            for u,v,p in edge[live_edge]: adj[int(u)].append(int(v))
            return adj
        contams_sampling = np.array([get_contam(make_adj(live_edge)) for live_edge in live_edges])
        
        # calcaulate contamination degree for each edge
        contams_edge = [np.sum(contams_sampling[dead_sample])/np.sum(dead_sample) if np.any(dead_sample) else float('inf') for dead_sample in dead_edges.T]
        
        # remove a edge
        min_v = float('inf')
        min_i = 0
        for i,v in enumerate(contams_edge):
            if min_v > v:
                min_v = v
                min_i = i
        mask.append([int(edge[min_i][0]),int(edge[min_i][1]),edge[min_i][2]])
        edge = np.delete(edge, min_i, axis=0)
        m-=1

        time_alg.append(time.time()-time_start)

    return mask, time_alg

In [43]:
def get_influence(adj, seed_idx):
    n = len(adj)
    visit = np.zeros(n, dtype=int)
    visit[seed_idx] = 1
    Q = deque(seed_idx)
    while Q:
        u = Q.popleft()
        for v in adj[u]:
            if not visit[v]:
                visit[v] = 1
                Q.append(v)
    return sum(visit)


def greedy_BPM(adj_list, seed_idx, del_edge_num, sampling_num=10000):
    time_start = time.time()
    time_alg = []
    mask = []
    
    # make edge list for efficient sampling
    n = len(adj_list)
    edge = []
    for u in range(n):
        for v,p in adj_list[u]: edge.append([u,v,p])
    edge = np.array(edge)
    m = len(edge)

    for _ in range(del_edge_num):        
        # sampling
        samplings = np.random.rand(sampling_num,m)
        live_edges = samplings<edge[:,2]
        dead_edges = ~live_edges
    
        # estimate influence for each sample
        def make_adj(live_edge):
            adj = [[] for _ in range(n)]
            for u,v,p in edge[live_edge]: adj[int(u)].append(int(v))
            return adj
        influence_sampling = np.array([get_influence(make_adj(live_edge),seed_idx) for live_edge in live_edges])
        
        # calculate influence for each edge
        influence_edge = [np.sum(influence_sampling[dead_sample])/np.sum(dead_sample) if np.any(dead_sample) else float('inf') for dead_sample in dead_edges.T]
        
        # remove a edge
        min_v = float('inf')
        min_i = 0
        for i,v in enumerate(influence_edge):
            if min_v > v:
                min_v = v
                min_i = i      
        mask.append([int(edge[min_i][0]),int(edge[min_i][1]),edge[min_i][2]])
        edge = np.delete(edge, min_i, axis=0)
        m-=1

        time_alg.append(time.time()-time_start)

    return mask, time_alg